<a href="https://colab.research.google.com/github/somitrasingh/deeplearning/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn


In [ ]:
class block(nn.Module):
  def __init__(self, in_channels, intermediate_channels, identity_downsample=None, stride=1):
    super(block, self).__init__()

    self.expansion = 4

    self.conv1 = nn.Conv2d(in_channels, intermediate_channels, kernel_size=1, stride=1, padding=0)
    self.bn1 = nn.BatchNorm2d(intermediate_channels)

    self.conv2 = nn.Conv2d(intermediate_channels, intermediate_channels, kernel_size=3, stride=stride, padding=1)
    self.bn2 = nn.BatchNorm2d(intermediate_channels)

    self.conv3 = nn.Conv2d(intermediate_channels, intermediate_channels*self.expansion, kernel_size=1, stride=1, padding=0)
    self.bn3 = nn.BatchNorm2d(intermediate_channels*self.expansion)


    self.relu = nn.ReLU()

    self.identity_downsample = identity_downsample

  def forward(self, x):
    identity = x.clone()
    x = self.relu(self.bn1(self.conv1(x)))
    x = self.relu(self.bn2(self.conv2(x)))
    x = self.bn3(self.conv3(x))

    if self.identity_downsample is not None:
      identity = self.identity_downsample(identity)

    x += identity
    x = self.relu(x)

    return x


In [ ]:
class ResNet(nn.Module):
  def __init__(self, block, layers, img_channels, num_classes):
    super(ResNet, self).__init__()
    self.in_channels = 64
    self.conv1 = nn.Conv2d(img_channels, out_channels=64, kernel_size=7, stride=2, padding=3)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU()
    self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

    self.layer1 = self._make_layer(block, layers[0], 64, stride=1)
    self.layer2 = self._make_layer(block, layers[1], 128, stride=2)
    self.layer3 = self._make_layer(block, layers[2], 256, stride=2)
    self.layer4 = self._make_layer(block, layers[3], 512, stride=2)

    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512 * 4, num_classes)


  def forward(self, x):
    x = self.pool1(self.relu(self.bn1(self.conv1(x))))
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)
    x = self.avgpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc(x)

    return x




  def _make_layer(self, block, layer, intermediate_channels, stride):
    layers = []

    identity_downsample = None
    if stride != 1 or self.in_channels != intermediate_channels*4:
      identity_downsample = nn.Sequential(
          nn.Conv2d(self.in_channels, intermediate_channels*4, kernel_size=1, stride=stride),
          nn.BatchNorm2d(intermediate_channels*4))

    layers.append(block(self.in_channels, intermediate_channels, identity_downsample, stride=stride))

    self.in_channels = intermediate_channels*4

    for i in range(layer-1):
      layers.append(block(self.in_channels, intermediate_channels))

    return nn.Sequential(*layers)



In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader

transform = transforms.Compose([
    transforms.Resize(128),
    transforms.CenterCrop(112),
    transforms.Lambda(lambda image: image.convert("RGB")),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

dataset = datasets.Caltech101(root='./data', download=True, transform=transform)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=500, shuffle=True, num_workers=0, pin_memory=True)

Files already downloaded and verified


In [ ]:
import torch.optim as optim
from torchsummary import summary

layers = [3,4,6,3]

model = ResNet(block, layers, 3, 101)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
summary(model, (3, 227, 227))
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 114, 114]           9,472
       BatchNorm2d-2         [-1, 64, 114, 114]             128
              ReLU-3         [-1, 64, 114, 114]               0
         MaxPool2d-4           [-1, 64, 57, 57]               0
            Conv2d-5           [-1, 64, 57, 57]           4,160
       BatchNorm2d-6           [-1, 64, 57, 57]             128
              ReLU-7           [-1, 64, 57, 57]               0
            Conv2d-8           [-1, 64, 57, 57]          36,928
       BatchNorm2d-9           [-1, 64, 57, 57]             128
             ReLU-10           [-1, 64, 57, 57]               0
           Conv2d-11          [-1, 256, 57, 57]          16,640
      BatchNorm2d-12          [-1, 256, 57, 57]             512
           Conv2d-13          [-1, 256, 57, 57]          16,640
      BatchNorm2d-14          [-1, 256,

In [ ]:
num_epochs = 20

best_acc = 0
for epoch in range(num_epochs):
    # Training phase
    model.train()
    total_loss = 0
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * labels.size(0)

        # Print progress
        if batch_idx % 100 == 0:
            print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

    avg_loss = total_loss / len(train_loader.dataset)

    # Evaluation phase
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Epoch {epoch}: Loss={avg_loss:.4f}, Accuracy={accuracy:.2f}%')

    # Save best model
    if accuracy > best_acc:
        best_acc = accuracy
        torch.save(model.state_dict(), 'best_model.pth')

Epoch: 0, Batch: 0, Loss: 0.0585
Epoch: 0, Batch: 100, Loss: 0.0257
Epoch 0: Loss=0.0275, Accuracy=63.82%
Epoch: 1, Batch: 0, Loss: 0.0017
Epoch: 1, Batch: 100, Loss: 0.0108
Epoch 1: Loss=0.0155, Accuracy=64.23%
Epoch: 2, Batch: 0, Loss: 0.0018
Epoch: 2, Batch: 100, Loss: 0.0054
Epoch 2: Loss=0.0225, Accuracy=64.69%
Epoch: 3, Batch: 0, Loss: 0.0045
Epoch: 3, Batch: 100, Loss: 0.0971
Epoch 3: Loss=0.0089, Accuracy=65.09%
Epoch: 4, Batch: 0, Loss: 0.0011
Epoch: 4, Batch: 100, Loss: 0.0010
Epoch 4: Loss=0.0024, Accuracy=65.73%
Epoch: 5, Batch: 0, Loss: 0.0005
Epoch: 5, Batch: 100, Loss: 0.0010
Epoch 5: Loss=0.0052, Accuracy=65.38%
Epoch: 6, Batch: 0, Loss: 0.0007
Epoch: 6, Batch: 100, Loss: 0.0018
Epoch 6: Loss=0.0061, Accuracy=64.86%
Epoch: 7, Batch: 0, Loss: 0.0009
Epoch: 7, Batch: 100, Loss: 0.2369
Epoch 7: Loss=0.1945, Accuracy=52.02%
Epoch: 8, Batch: 0, Loss: 0.5096
Epoch: 8, Batch: 100, Loss: 0.2966
Epoch 8: Loss=0.4463, Accuracy=58.41%
Epoch: 9, Batch: 0, Loss: 0.1962
Epoch: 9, Bat

In [ ]:
num_epochs = 20
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()
for epoch in range(num_epochs):
  total_loss = 0
  for batch_idx, (images, labels) in enumerate(train_loader):
    images, labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    y_hat = model(images)
    loss = criterion(y_hat, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item() * labels.size(0)

    if batch_idx % 100 == 0:
            print(f'Epoch: {epoch}, Batch: {batch_idx}, Loss: {loss.item():.4f}')

  avg_loss = total_loss/len(train_loader.dataset)
  print(f"Loss after {epoch+1} epochs: {avg_loss}")


Epoch: 0, Batch: 0, Loss: 4.8311
Epoch: 0, Batch: 100, Loss: 3.8643
Loss after 1 epochs: 5.121669146808865
Epoch: 1, Batch: 0, Loss: 4.2141


KeyboardInterrupt: 

In [ ]:
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    images = (images - images.mean())/images.std()
    y_hat = model(images)
    _, y_hat = torch.max(y_hat, 1)
    total += labels.size(0)
    correct += (y_hat == labels).sum().item()

  accuracy = 100 * correct / total
  print(f'Epoch {epoch}: Loss={avg_loss:.4f}, Accuracy={accuracy:.2f}%')

  # Save best model
  if accuracy > best_acc:
      best_acc = accuracy
      torch.save(model.state_dict(), 'best_model.pth')
